In [ ]:
#GPU check
!nvidia-smi

Sat Apr  2 14:31:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    73W / 149W |   1711MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Install the latest release of Haystack in your own environment
#! pip install farm-haystack
# Install the latest master of Haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-3mqr6ob_/farm-haystack_9a114f0026c841999cc9775a4a83bb93
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-3mqr6ob_/farm-haystack_9a114f0026c841999cc9775a4a83bb93
  Resolved https://github.com/deepset-ai/haystack.git to commit ae712fe6bf087c717f3e38e4e87d2347165fc12b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http, print_answers
from haystack.nodes import FARMReader, TransformersReader

In [ ]:
from haystack.utils import launch_es
launch_es()

WARNING - haystack.utils.doc_store -  Tried to start Elasticsearch through Docker but this failed. It is likely that there is already an existing Elasticsearch instance running. 


In [ ]:
# In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)
# wait until ES has started
! sleep 30

In [ ]:
# Connect to Elasticsearch

from haystack.document_stores import ElasticsearchDocumentStore

document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

In [ ]:

dicts = convert_files_to_docs(dir_path="/content/data", clean_func=clean_wiki_text, split_paragraphs=True)
print(dicts[:3])
document_store.write_documents(dicts)

INFO - haystack.utils.preprocessing -  Converting /content/data/gs_10k_2021.txt


[<Document: {'content': 'Goldman Sachs is a leading global financial institution that delivers a broad\nrange of financial services across investment banking, securities, investment\nmanagement and consumer banking to a large and diversified client base that\nincludes corporations, financial institutions, governments and individuals.\nOur purpose is to advance sustainable economic growth and financial\nopportunity. Our goal, reflected in our\ninitiative, is to deliver the full range of our services and expertise to\nsupport our clients in a more accessible, comprehensive and efficient manner,\nacross businesses and product areas.\nWhen we use the terms “Goldman Sachs,” “we,” “us” and “our,” we mean The\nGoldman Sachs Group, Inc. (Group Inc. or parent company), a Delaware\ncorporation, and its consolidated subsidiaries. When we use the term “our\nsubsidiaries,” we mean the consolidated subsidiaries of Group Inc. References\nfor the year ended December 31, 2021. All references to 2021, 2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from haystack.nodes import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)

In [ ]:
# Load a  local model or any of the QA models on
# Hugging Face's model hub (https://huggingface.co/models)
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find deepset/roberta-base-squad2 locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Loaded deepset/roberta-base-squad2
INFO - haystack.modeling.logger -  ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.infer -  Got ya 2 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0     0  
INFO - haystack.modeling.infer -  /w\   /w\ 
INFO - haystack.modeling.infer -  /'\   / \ 


In [ ]:
from haystack.pipelines import ExtractiveQAPipeline
pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
prediction = pipe.run(
    query="Who is the president of Goldman Sachs?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)
print_answers(prediction, details="medium")

Inferencing Samples:   0%|          | 0/62 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/haystack/modeling/model/prediction_head.py:485: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  2.47 Batches/s]


Query: Who is the president of Goldman Sachs?
Answers:
[   {   'answer': 'Mr. Waldron',
        'context': 'd\n'
                   'of the Investment Banking Division from July 2006 to '
                   'December 2016.\n'
                   'Mr. Waldron has been President and Chief Operating Officer '
                   'since October 2018.\n'
                   'o',
        'score': 0.7701091766357422},
    {   'answer': 'Mr. Berlinski',
        'context': 'been\n'
                   'appointed by, and serve at the pleasure of, Group Inc.’s '
                   'Board.\n'
                   'Mr. Berlinski has been Global Treasurer since October '
                   '2021; he also serves as\n'
                   'Chi',
        'score': 0.6145103573799133},
    {   'answer': 'David Solomon',
        'context': 'K.\n'
                   'February 24, 2022EX-31.1 15 d192225dex311.htm EX-31.1  '
                   'EX-31.1\n'
                   'I, David Solomon, certify that:\n'
    

In [ ]:
prediction = pipe.run(
    query="How goldman sachs exposed to credit risk?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)
print_answers(prediction, details="medium")

Inferencing Samples:   0%|          | 0/62 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/haystack/modeling/model/prediction_head.py:485: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.27 Batches/s]


Query: How goldman sachs exposed to credit risk?
Answers:
[   {   'answer': 'client transactions in OTC\n'
                  'derivatives and loans and lending commitments',
        'context': 'osure to credit risk comes mostly from client transactions '
                   'in OTC\n'
                   'derivatives and loans and lending commitments. Credit risk '
                   'also comes from\n'
                   'cash plac',
        'score': 0.624679684638977},
    {   'answer': 'increased exposures',
        'context': ' December 2021, unchanged compared with December\n'
                   '2020, reflecting increased exposures, offset by lower '
                   'levels of volatility.\n'
                   'This was driven by increa',
        'score': 0.6181199252605438},
    {   'answer': 'many different industries and counterparties',
        'context': 'litical factors. These activities expose the firm\n'
                   'to many different industries and counterpartie

In [ ]:
prediction = pipe.run(
    query="When will business be affected?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)
print_answers(prediction, details="medium")

Inferencing Samples:   0%|          | 0/62 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/haystack/modeling/model/prediction_head.py:485: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  5.12 Batches/s]


Query: When will business be affected?
Answers:
[   {   'answer': 'Within 30 Business Days',
        'context': 'on of the Transfer\n'
                   'Restrictions on Shares at Risk will be void. Within 30 '
                   'Business Days after the\n'
                   'Transferability Date listed on your Award Statement ',
        'score': 0.644675225019455},
    {   'answer': 'if we are unable to hire and retain',
        'context': 'wise adversely\n'
                   'Our businesses would be adversely affected if we are '
                   'unable to hire and retain\n'
                   'Our performance is largely dependent on the talents and ',
        'score': 0.5841364562511444},
    {   'answer': 'After the\n'
                  'expiration of the trial period, the trial edition will stop '
                  'running',
        'context': 'plus any permitted extension period. After the\n'
                   'expiration of the trial period, the trial edition will '
   

In [ ]:
prediction = pipe.run(
    query="Where are the competitions from?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)
print_answers(prediction, details="medium")

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/haystack/modeling/model/prediction_head.py:485: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.12 Batches/s]


Query: Where are the competitions from?
Answers:
[   {   'answer': 'emerging and growth markets',
        'context': 'sses and our technology initiatives. This is\n'
                   'also the case in emerging and growth markets, where we are '
                   'often competing for\n'
                   'qualified employees with e',
        'score': 0.7295897901058197},
    {   'answer': 'within the financial services industry',
        'context': 'MAN SACHS GROUP, INC. AND SUBSIDIARIES\n'
                   'Competition from within the financial services industry '
                   'and from businesses\n'
                   'outside the financial services indu',
        'score': 0.39459826052188873},
    {   'answer': 'other asset managers',
        'context': 'ower than expected due to,\n'
                   'among other factors, competition from other asset '
                   'managers, changes in\n'
                   'investment preferences and changes in economic or ma',

In [ ]:
prediction = pipe.run(
    query="Who is the CFO?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)
print_answers(prediction, details="medium")

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/haystack/modeling/model/prediction_head.py:485: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  2.47 Batches/s]


Query: Who is the CFO?
Answers:
[   {   'answer': 'Mr. Coleman',
        'context': 'al Equities Trading and Execution Services from September '
                   '2016 to May\n'
                   'Mr. Coleman has been Chief Financial Officer since January '
                   '2022. He had\n'
                   'previous',
        'score': 0.8438294231891632},
    {   'answer': 'Mr. Berlinski',
        'context': 'been\n'
                   'appointed by, and serve at the pleasure of, Group Inc.’s '
                   'Board.\n'
                   'Mr. Berlinski has been Global Treasurer since October '
                   '2021; he also serves as\n'
                   'Chi',
        'score': 0.5601659417152405},
    {   'answer': 'David',
        'context': ' 10-K.\n'
                   'February 24, 2022EX-31.1 15 d192225dex311.htm EX-31.1  '
                   'EX-31.1\n'
                   'I, David Solomon, certify that:\n'
                   'I have reviewed this Annual Report